Q-2. Take any pdf and your task is to extract the text from that pdf and store it in a
csv file and then you need to find the most repeated word in that pdf.

In [8]:
pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.4 MB/s eta 0:00:00


In [12]:
import PyPDF2
import pandas as pd

# Open the PDF file
pdf_file = "/content/Data Science.pdf"
with open(pdf_file, "rb") as file:
    pdf_reader = PyPDF2.PdfReader(file)

    # Extract text from each page of the PDF
    text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text += page.extract_text()

# Store the extracted text in a CSV file
csv_file = "/content/sample_data/Data Science.csv"
data = {"Text": [text]}
df = pd.DataFrame(data)
df.to_csv(csv_file, index=False)

# Find the most repeated word
words = text.split()
word_count = {}
for word in words:
    if word in word_count:
        word_count[word] += 1
    else:
        word_count[word] = 1

most_repeated_word = max(word_count, key=word_count.get)

print("Most repeated word:", most_repeated_word)


Most repeated word: the


Q-3. from question 2, As you got the CSV and now you need perform key word
extraction from that csv file and do the Topic modeling


In [14]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

In [15]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [17]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [20]:
# Read the CSV file
csv_file = "/content/sample_data/Data Science.csv"
df = pd.read_csv(csv_file)

# Combine all text into a single string
text = ' '.join(df['Text'].astype(str))

# Tokenize the text into individual words
tokens = word_tokenize(text)

# Remove stopwords and non-alphabetic characters
stop_words = set(stopwords.words('english'))
tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]

# Lemmatize the tokens
lemmatizer = WordNetLemmatizer()
tokens = [lemmatizer.lemmatize(token) for token in tokens]

# Create a dictionary from the tokens
dictionary = corpora.Dictionary([tokens])

# Create a bag-of-words representation of the text
corpus = [dictionary.doc2bow(tokens)]

# Perform topic modeling using LDA
num_topics = 5  # Set the number of topics
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary)

# Get the most significant words for each topic
topic_keywords = []
for topic_id in range(num_topics):
    words = lda_model.show_topic(topic_id, topn=5)
    topic_keywords.append([word for word, _ in words])

# Calculate coherence score for the model
coherence_model = CoherenceModel(model=lda_model, texts=[tokens], dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

# Print the topic keywords and coherence score
for topic_id, keywords in enumerate(topic_keywords):
    print(f"Topic {topic_id + 1}: {keywords}")
print("Coherence Score:", coherence_score)


Topic 1: ['e', 'n', 'b', 'r', 'q']
Topic 2: ['e', 'n', 'r', 'q', 'k']
Topic 3: ['q', 'r', 'e', 'n', 'company']
Topic 4: ['e', 'r', 'n', 'q', 'b']
Topic 5: ['e', 'r', 'n', 'q', 'b']
Coherence Score: 0.2748554087892231


Q-4. Take any text file and now your task is to Text Summarization without using
hugging transformer library


In [24]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np

def read_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

def preprocess_text(text):
    sentences = sent_tokenize(text)
    stop_words = set(stopwords.words('english'))
    cleaned_sentences = []
    
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        words = [word.lower() for word in words if word.isalpha()]
        words = [word for word in words if word not in stop_words]
        cleaned_sentences.append(words)
    
    return cleaned_sentences

def calculate_sentence_similarity(sentence1, sentence2):
    sentence1 = [word.lower() for word in sentence1]
    sentence2 = [word.lower() for word in sentence2]
    all_words = list(set(sentence1 + sentence2))
    
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    for word in sentence1:
        vector1[all_words.index(word)] += 1
    
    for word in sentence2:
        vector2[all_words.index(word)] += 1
    
    return 1 - cosine_distance(vector1, vector2)

def create_similarity_matrix(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = calculate_sentence_similarity(sentences[i], sentences[j])
    
    return similarity_matrix

def generate_summary(file_path, num_sentences):
    text = read_text(file_path)
    cleaned_sentences = preprocess_text(text)
    similarity_matrix = create_similarity_matrix(cleaned_sentences)
    
    # PageRank algorithm to rank sentences based on similarity
    scores = np.zeros(len(cleaned_sentences))
    damping_factor = 0.85
    epsilon = 0.001
    
    while True:
        prev_scores = np.copy(scores)
        
        for i in range(len(cleaned_sentences)):
            summation = 0
            
            for j in range(len(cleaned_sentences)):
                if i != j:
                    summation += (similarity_matrix[i][j] / np.sum(similarity_matrix[j])) * scores[j]
            
            scores[i] = (1 - damping_factor) + damping_factor * summation
        
        if np.sum(np.abs(prev_scores - scores)) < epsilon:
            break
    
    ranked_sentences = [(score, sentence) for score, sentence in zip(scores, cleaned_sentences)]
    ranked_sentences.sort(key=lambda x: x[0], reverse=True)
    summary_sentences = [sentence for _, sentence in ranked_sentences[:num_sentences]]
    summary = ' '.join([' '.join(sentence) for sentence in summary_sentences])
    
    return summary

# Provide the file path and the desired number of summary sentences
file_path = "/content/fsds assessment.txt"
num_sentences = 3
summary = generate_summary(file_path, num_sentences)

print("Summary:")
print(summary)


KeyboardInterrupt: ignored

Q-5. Now you need build your own language detection with the fast Text model
by Facebook and


In [25]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4393395 sha256=d3790507a865b8e491709d7044c674a20dff5b8df4f3585efa0ba49457664176
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [1]:
import fasttext

# Path to the training data file
train_data = "/content/fsds assessment.txt"

# Train the model
model = fasttext.train_supervised(input=train_data, lr=1.0, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='softmax')


In [ ]:
# Example text to predict its language
text = "the, hindi"

# Predict the language
prediction = model.predict(text)
language = prediction[0][0].split("__")[-1]

print("Predicted Language:", language)
